In [47]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np
import random
from ast import literal_eval


In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 100
num_workers = 0
N = 1370
annotation_train = 'data/data_se/data_train.txt'
annotation_val = 'data/data_se/data_test.txt'
epoch = 100
Loss = nn.CrossEntropyLoss()
lr = 0.0001
best_Loss_regression = 10000
best_model_wts = None
best_model_ind = 0
best_model_acc = 0

In [49]:
class MyDataset(Dataset):
    def __init__(self, annotation, N):
        self.landmarks_frame = pd.read_csv(annotation)
        self.N = N

    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        i_data = literal_eval(self.landmarks_frame.iloc[idx, 1])
        i = random.choice(i_data)
        data = np.zeros(self.N,dtype = np.float32)
        for d in i_data:
            data[d] += 1
        data[i] -= 1
        return data, i
train_dataset = MyDataset(annotation_train, N)
val_dataset = MyDataset(annotation_val, N)

In [50]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=False)

In [51]:
class PerceptronNet(nn.Module):
    def __init__(self, N):
        super(PerceptronNet, self).__init__()
        self.L1 = nn.Linear(N, 2*N)
        self.L2 = nn.Linear(2*N, 2*N)
        self.L3 = nn.Linear(2*N, N)
        self.relu = nn.ReLU()
        self.Softmax = nn.Softmax(1)
    def forward(self,x):
        x = self.relu(self.L1(x))
        x = self.relu(self.L2(x))
        x = self.relu(self.L3(x))
        return x
net = PerceptronNet(N).to(device)
optimizer = optim.AdamW(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=5)

In [ ]:
for iter in range(epoch):
    for dataloader, phase in zip([train_dataloader, val_dataloader], ["T", "V"]):
        eposh_loss = 0
        if phase == "T": net.train()
        if phase == "V": net.eval()
        for data, laibl in dataloader:
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == "T"):
                out = net(data.to(device))
                # print(out.shape, laibl.shape)
                loss = Loss(out, laibl.to(device))
                # print(out, laibl, loss)
                if phase == 'T':
                    # Вычислить градиенты
                    loss.backward()
                    # Обновить веса
                    optimizer.step()
                eposh_loss+=loss.item()
        if phase == "T": 
            print(f"{iter} {phase} {eposh_loss / len(dataloader):0.3f}", end=" ")
        else:
            scheduler.step(eposh_loss)
            print(f"{phase} {eposh_loss / len(dataloader):0.3f}")
            if eposh_loss < best_Loss_regression:
                best_model_wts = net.state_dict()
                best_model_ind = iter
                best_Loss_regression = eposh_loss
net.load_state_dict(best_model_wts)
torch.save(best_model_wts, 'model.tar')
print(best_model_ind, best_Loss_regression / len(val_dataloader))

In [53]:
net.load_state_dict(best_model_wts)
torch.save(best_model_wts, 'model.tar')
print(best_model_ind, best_Loss_regression / len(val_dataloader))

24 6.810856623152282


In [69]:
n = 0
trye = 0
trye_3 = 0
trye_5 = 0
trye_10 = 0
trye_50 = 0
trye_500 = 0
data_test = pd.read_csv(annotation_val)
for d in data_test['name_id'].values:
    mas = literal_eval(d)
    for i, m in enumerate(mas):
        buf = np.zeros((N),dtype = np.float32)
        for el in [*mas[:i],*mas[i+1:]]:
            buf[el] +=1
        buf[m] -=1
        inpyt = torch.tensor(buf).to(device)
        out = net(inpyt)
        trye += mas[i] == out.argmax()
        trye_3 += mas[i] in out.argsort(descending=True)[:3]
        trye_5 += mas[i] in out.argsort(descending=True)[:5]
        trye_10 += mas[i] in out.argsort(descending=True)[:10]
        trye_50 += mas[i] in out.argsort(descending=True)[:50]
        trye_500 += mas[i] in out.argsort(descending=True)[:500]
        n+=1
print('Acc top 1', trye / n)
print('Acc top 3',trye_3 / n)
print('Acc top 5',trye_5 / n)
print('Acc top 10',trye_10 / n)
print('Acc top 50',trye_50 / n)
print('Acc top 500',trye_500 / n)

Acc top 1 tensor(0.0992, device='cuda:0')
Acc top 3 0.10831234256926953
Acc top 5 0.11122635452165754
Acc top 10 0.11537511730132859
Acc top 50 0.16666666666666666
Acc top 500 0.5693436064602163
